# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('ec_freq.npz') 

# Access the concatenated features from the .npz file
X_freq = data['X_frequency']

# Close the loaded file
data.close()

#print the merged array
X_freq

array([[[ 1.46584672e-13,  6.45390788e-21,  1.15009007e-25, ...,
          1.04873571e+00,  1.75910640e+00,  9.32599360e-06],
        [ 8.03138316e-14,  1.93266859e-21,  6.30700474e-26, ...,
          2.39037981e-01,  6.38193078e-01, -6.60421011e-06],
        [ 1.71122109e-13,  2.75453640e-23,  2.63616577e-25, ...,
          3.38987332e-01,  5.25518596e-01, -5.34106217e-06],
        ...,
        [ 1.52121787e-13,  8.47677259e-23,  2.30035066e-25, ...,
          2.90477535e-01,  3.20355828e-01, -9.29930152e-07],
        [ 1.50910431e-13,  4.19486197e-22,  2.48974250e-25, ...,
          5.94779033e-02,  2.76903208e-01,  1.20278818e-07],
        [ 4.84596455e-13,  4.15998616e-21,  2.19098742e-24, ...,
          2.88563512e-01,  8.60427949e-01,  3.48740780e-06]],

       [[ 2.12847299e-12,  7.43448561e-21,  5.54611187e-23, ...,
          6.50319496e-02,  1.25496815e-01, -8.67524675e-06],
        [ 3.88678177e-13,  3.17041833e-21,  1.55235289e-24, ...,
          1.08033075e-01,  3.88190787e

In [3]:
data = np.load('eyes_close_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('eyes_close_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_freq.shape, Y.shape, groups_array.shape)

(17325, 20, 20) (17325,) (17325,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_freq, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_freq[train_index], X_freq[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
61/61 [==============================] - 12s 49ms/step - loss: 0.4505 - accuracy: 0.8019 - val_loss: 0.4786 - val_accuracy: 0.8182
Epoch 2/200
61/61 [==============================] - 1s 23ms/step - loss: 0.3451 - accuracy: 0.8409 - val_loss: 0.4171 - val_accuracy: 0.8442
Epoch 3/200
61/61 [==============================] - 1s 22ms/step - loss: 0.3149 - accuracy: 0.8582 - val_loss: 0.3404 - val_accuracy: 0.8373
Epoch 4/200
61/61 [==============================] - 1s 24ms/step - loss: 0.2927 - accuracy: 0.8685 - val_loss: 0.3198 - val_accuracy: 0.8621
Epoch 5/200
61/61 [==============================] - 1s 24ms/step - loss: 0.2731 - accuracy: 0.8773 - val_loss: 0.2881 - val_accuracy: 0.8707
Epoch 6/200
61/61 [==============================] - 1s 24ms/step - loss: 0.2661 - accuracy: 0.8815 - val_loss: 0.2631 - val_accuracy: 0.8857
Epoch 7/200
61/61 [==============================] - 1s 22ms/step - loss: 0.2546 - accuracy: 0.8847 - val_loss: 0.2717 - val_accuracy: 0.882

In [18]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [19]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9215,0.9207,0.9249,0.9228,0.9215,0.9180,0.9249,0.8430,0.8430
1,0.9400,0.9429,0.9386,0.9407,0.9400,0.9415,0.9386,0.8800,0.8800
2,0.9169,0.9088,0.9295,0.9190,0.9167,0.9040,0.9295,0.8337,0.8339
3,0.9163,0.9124,0.9238,0.9180,0.9162,0.9087,0.9238,0.8326,0.8327
4,0.9192,0.9204,0.9204,0.9204,0.9192,0.9180,0.9204,0.8384,0.8384
5,0.9296,0.9386,0.9215,0.9300,0.9297,0.9379,0.9215,0.8591,0.8593
6,0.9140,0.9110,0.9204,0.9157,0.9139,0.9074,0.9204,0.8279,0.8279
7,0.9192,0.9213,0.9192,0.9203,0.9192,0.9191,0.9192,0.8383,0.8383
8,0.9249,0.9280,0.9238,0.9259,0.9250,0.9261,0.9238,0.8499,0.8499
9,0.9342,0.9432,0.9261,0.9346,0.9343,0.9426,0.9261,0.8684,0.8685


In [20]:
metrics_df.round(4).to_csv('FREQ_EC_CNN_LSTM.csv', index = False)

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
result = pd.read_csv("FREQ_EC_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9215,0.9207,0.9249,0.9228,0.9215,0.9180,0.9249,0.8430,0.8430
1,0.9400,0.9429,0.9386,0.9407,0.9400,0.9415,0.9386,0.8800,0.8800
2,0.9169,0.9088,0.9295,0.9190,0.9167,0.9040,0.9295,0.8337,0.8339
3,0.9163,0.9124,0.9238,0.9180,0.9162,0.9087,0.9238,0.8326,0.8327
4,0.9192,0.9204,0.9204,0.9204,0.9192,0.9180,0.9204,0.8384,0.8384
5,0.9296,0.9386,0.9215,0.9300,0.9297,0.9379,0.9215,0.8591,0.8593
6,0.9140,0.9110,0.9204,0.9157,0.9139,0.9074,0.9204,0.8279,0.8279
7,0.9192,0.9213,0.9192,0.9203,0.9192,0.9191,0.9192,0.8383,0.8383
8,0.9249,0.9280,0.9238,0.9259,0.9250,0.9261,0.9238,0.8499,0.8499
9,0.9342,0.9432,0.9261,0.9346,0.9343,0.9426,0.9261,0.8684,0.8685


In [23]:
(result.mean()*100).round(2)

Accuracy       92.36
Precision      92.47
Recall         92.48
F1             92.47
Sensitivity    92.36
Specificity    92.23
ROC_AUC        92.48
Kappa          84.71
MCC            84.72
dtype: float64

In [17]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])